# Execute log-odds
## Here we use the calculated log odds between greyroads corpus and civility data to score each term in Eurovoc 

In [2]:
import os
import csv
import pandas as pd
import spacy
from dtm_toolkit.preprocessing import Preprocessing
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

nlp = spacy.load("en_core_web_sm")

logodds_res_path = "log-odds/greyroads_logodds.tsv"
greyroads_corpus_path = "log-odds/greyroads_corpus.tsv"
with open(logodds_res_path) as fp:
    reader = csv.reader(fp, delimiter='\t')
    logodds = {k:float(v.strip("\n")) for k,v in reader}
with open(greyroads_corpus_path) as fp:
    reader = csv.reader(fp, delimiter='\t')
    greyroads = {k:float(v.strip("\n")) for k,v in reader}
eurovoc = pd.read_csv("../../static/eurovoc_export_en.csv")

In [8]:
labels = eurovoc['MT'].tolist()
# put in correct form for ingestion into dtm creator
docs = nlp.pipe(eurovoc['TERMS (PT-NPT)'].tolist(), n_process=11, batch_size=256)
preproc = Preprocessing(docs)
preproc_terms = preproc.preprocess(ngrams=True)

adding ngrams...
doc 0
doc 1
doc 2
doc 3
doc 4
doc 5
doc 6
doc 7
doc 8
doc 9
doc 10
doc 11
doc 12
doc 13
doc 14
doc 15
doc 16
doc 17
doc 18
doc 19
doc 20
doc 21
doc 22
doc 23
doc 24
doc 25
doc 26
doc 27
doc 28
doc 29
doc 30
doc 31
doc 32
doc 33
doc 34
doc 35
doc 36
doc 37
doc 38
doc 39
doc 40
doc 41
doc 42
doc 43
doc 44
doc 45
doc 46
doc 47
doc 48
doc 49
doc 50
doc 51
doc 52
doc 53
doc 54
doc 55
doc 56
doc 57
doc 58
doc 59
doc 60
doc 61
doc 62
doc 63
doc 64
doc 65
doc 66
doc 67
doc 68
doc 69
doc 70
doc 71
doc 72
doc 73
doc 74
doc 75
doc 76
doc 77
doc 78
doc 79
doc 80
doc 81
doc 82
doc 83
doc 84
doc 85
doc 86
doc 87
doc 88
doc 89
doc 90
doc 91
doc 92
doc 93
doc 94
doc 95
doc 96
doc 97
doc 98
doc 99
doc 100
doc 101
doc 102
doc 103
doc 104
doc 105
doc 106
doc 107
doc 108
doc 109
doc 110
doc 111
doc 112
doc 113
doc 114
doc 115
doc 116
doc 117
doc 118
doc 119
doc 120
doc 121
doc 122
doc 123
doc 124
doc 125
doc 126
doc 127
doc 128
doc 129
doc 130
doc 131
doc 132
doc 133
doc 134
doc 135
doc 1

In [10]:
preproc_terms = ["_".join(x[0]) for x in preproc_terms]
assert len(preproc_terms) == len(labels)
preproc_terms

['print',
 '',
 '',
 '',
 'aa',
 'aams_country',
 'aar',
 'aau',
 'abb',
 'abm_agreement',
 'acer',
 'achpr',
 'acj',
 'acjhr',
 'acp_country',
 'acp_ec_committee_ambassadors',
 'acp_ec_convention',
 'acp_ec_council',
 'acp_ec_council_ministers',
 'acp_ec_joint_assembly',
 'acp_ec_joint_committee',
 'acp_ec_institution',
 'acp_eec_consultative_assembly',
 'acp_eu_committee_ambassadors',
 'acp_eu_convention',
 'acp_eu_council_ministers',
 'acp_eu_joint_assembly',
 'acp_eu_joint_committee',
 'acp_eu_cooperation',
 'acp_eu_institution',
 'acp_eu_relationship',
 'acs',
 'ad_function_group',
 'adb',
 'adb_group',
 'adn_agreement',
 'adnr_agreement',
 'adr',
 'adr',
 'adr_agreement',
 'aetr_agreement',
 'afsj',
 'agcc',
 'agp',
 'aids',
 'aif',
 'aifm',
 'aladi',
 'aladi_country',
 'amco',
 'amr',
 'amu',
 'apdc',
 'apec',
 'apec',
 'apec_country',
 'apl',
 'apo',
 'appu',
 'artemis_joint_undertaking',
 'asean',
 'asean_country',
 'asp',
 'aspac',
 'ast_function_group',
 'atm',
 'atp_agreeme

In [12]:
aeo_path = os.path.join(os.environ['ROADMAP_SCRAPER'], "DTM", "greyroads_aeo_all.csv")
ieo_path = os.path.join(os.environ['ROADMAP_SCRAPER'], "DTM", "greyroads_ieo_all.csv")
journals_path = os.path.join(os.environ['ROADMAP_SCRAPER'], "journals", "journals_energy_policy_applied_energy_all_years_abstract.csv")
aeo = pd.read_csv(aeo_path)
ieo = pd.read_csv(ieo_path)
journals = pd.read_csv(journals_path)

In [13]:
aeo_corpus_raw = aeo['para_text'].dropna().tolist()
ieo_corpus_raw = ieo['para_text'].dropna().tolist()
journals_corpus_raw = journals['section_txt'].dropna().tolist()
merged_corpus = nlp.pipe(aeo_corpus_raw + ieo_corpus_raw + journals_corpus_raw, n_process=11, batch_size=256)
preproc = Preprocessing(merged_corpus)
preproc_corpus = preproc.preprocess(ngrams=True)

adding bigrams...
para 0
para 1000
para 2000
para 3000
para 4000
para 5000
para 6000
para 7000
para 8000
para 9000
para 10000
para 11000
para 12000
para 13000
para 14000
para 15000
para 16000
para 17000
para 18000
para 19000
para 20000
para 21000
para 22000
para 23000
para 24000
para 25000
para 26000
para 27000
para 28000


In [14]:
preproc_docs = preproc.get_merged_sents()

In [15]:
no_matches = set()
has_matches = Counter()

def score_eurovoc_labels():
    c = {}
    for i, doc in enumerate(preproc_docs):
        if i % 100 == 0:
            print(f"doc {i}")
        for tok in doc:
            toks = []
            # if the token has a logodds score
            if tok in logodds:
                score = logodds[tok]
                # find all full-string matches for this token in eurovoc
                res_list = list(filter(lambda x: preproc_terms[x] == tok, range(len(preproc_terms))))
                if res_list:
                    toks.extend([tok for _ in range(len(res_list))])
                    # add to set of matched tokens
                    for ind in res_list:
                        label = labels[ind]
                        # update eurovoc label with the logodds score of this token
                        if label in c:
                            c[label].append(score)
                        else:
                            c[label] = [score]
                else:
                    # add to set of not matched tokens
                    no_matches.add(tok)
            has_matches.update(toks)
    final_scores = Counter({label:np.median(np.array(scorelist)) for label, scorelist in c.items()})
    return final_scores

In [16]:
ev_scores = score_eurovoc_labels()

doc 0
doc 100
doc 200
doc 300
doc 400
doc 500
doc 600
doc 700
doc 800
doc 900
doc 1000
doc 1100
doc 1200
doc 1300
doc 1400
doc 1500
doc 1600
doc 1700
doc 1800
doc 1900
doc 2000
doc 2100
doc 2200
doc 2300
doc 2400
doc 2500
doc 2600
doc 2700
doc 2800
doc 2900
doc 3000
doc 3100
doc 3200
doc 3300
doc 3400
doc 3500
doc 3600
doc 3700
doc 3800
doc 3900
doc 4000
doc 4100
doc 4200
doc 4300
doc 4400
doc 4500
doc 4600
doc 4700
doc 4800
doc 4900
doc 5000
doc 5100
doc 5200
doc 5300
doc 5400
doc 5500
doc 5600
doc 5700
doc 5800
doc 5900
doc 6000
doc 6100
doc 6200
doc 6300
doc 6400
doc 6500
doc 6600
doc 6700
doc 6800
doc 6900
doc 7000
doc 7100
doc 7200
doc 7300
doc 7400
doc 7500
doc 7600
doc 7700
doc 7800
doc 7900
doc 8000
doc 8100
doc 8200
doc 8300
doc 8400
doc 8500
doc 8600
doc 8700
doc 8800
doc 8900
doc 9000
doc 9100
doc 9200
doc 9300
doc 9400
doc 9500
doc 9600
doc 9700
doc 9800
doc 9900
doc 10000
doc 10100
doc 10200
doc 10300
doc 10400
doc 10500
doc 10600
doc 10700
doc 10800
doc 10900
doc 11000
do

In [27]:
import pickle
with open("ev_label_logodds_scores.pickle", "wb+") as fp:
    pickle.dump(ev_scores, fp)

# The list of log-odds ranked EuroVoc labels is displayed below.
## See section 4.1 of paper.

In [4]:
import pickle
with open("ev_label_logodds_scores.pickle", "rb") as fp:
    data = pickle.load(fp)
data.most_common()

[('6621 electrical and nuclear industries', 100.52569263144737),
 ('6616 oil industry', 85.19232035725217),
 ('6611 coal and mining industries', 83.3305908599593),
 ('6406 production', 77.68822200428787),
 ('2451 prices', 77.18119846837699),
 ('6831 building and public works', 73.20855634681462),
 ('2026 consumption', 71.8321011770654),
 ('6606 energy policy', 70.48089853745519),
 ('6836 wood industry', 59.09702726406576),
 ('4026 accounting', 50.30161369082129),
 ('4816 land transport', 45.13751698104219),
 ('2016 trade', 44.82188511177203),
 ('6416 research and intellectual property', 43.402616778118926),
 ('7621 world organisations', 42.359943169088965),
 ('4811 organisation of transport', 41.74437176032781),
 ('6626 soft energy', 41.53992247785092),
 ('2036 distributive trades', 41.09143307719164),
 ('2006 trade policy', 40.713784433864106),
 ('6826 electronics and electrical engineering', 40.254325181877036),
 ('2446 taxation', 37.607133990413345),
 ('1631 economic analysis', 37.0